<a href="https://colab.research.google.com/github/iamprakashom/Credit-Scoring/blob/master/Credit%20Scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd, numpy as np
import datetime

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve, auc, confusion_matrix
from sklearn.model_selection import GridSearchCV

import sklearn
from sklearn_pandas import DataFrameMapper

import matplotlib.pyplot as plt

Colaboratory allows you to execute TensorFlow code in your browser with a single click. The example below adds two matrices.

$\begin{bmatrix}
  1. & 1. & 1. \\
  1. & 1. & 1. \\
\end{bmatrix} +
\begin{bmatrix}
  1. & 2. & 3. \\
  4. & 5. & 6. \\
\end{bmatrix} =
\begin{bmatrix}
  2. & 3. & 4. \\
  5. & 6. & 7. \\
\end{bmatrix}$

In [4]:
IN_DATAFILE='https://s3.amazonaws.com/rorodata-datasets/lending-club-data.csv'
loans = pd.read_csv(IN_DATAFILE, infer_datetime_format = True)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Colaboratory includes widely used libraries like [matplotlib](https://matplotlib.org/), simplifying visualization.

In [5]:
loans.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,sub_grade_num,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none
0,1077501,1296599,5000,5000,4975,36 months,10.65,162.87,B,B2,...,0.4,1.0,1.0,1.0,0,8.14350,20141201T000000,1,1,1
1,1077430,1314167,2500,2500,2500,60 months,15.27,59.83,C,C4,...,0.8,1.0,1.0,1.0,1,2.39320,20161201T000000,1,1,1
2,1077175,1313524,2400,2400,2400,36 months,15.96,84.33,C,C5,...,1.0,1.0,1.0,1.0,0,8.25955,20141201T000000,1,1,1
3,1076863,1277178,10000,10000,10000,36 months,13.49,339.31,C,C1,...,0.2,1.0,1.0,1.0,0,8.27585,20141201T000000,0,1,1
4,1075269,1311441,5000,5000,5000,36 months,7.90,156.46,A,A4,...,0.8,1.0,1.0,1.0,0,5.21533,20141201T000000,1,1,1


In [8]:
loans.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'is_inc_v', 'issue_d',
       'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose', 'title',
       'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'not_compliant', 'status', 'inactive_loans', 'bad_loans',
       'emp_length_num', 'grade_num', 'sub_gra

**Choosing Relevant Features**
- The dataframe loans has 68 columns of data. However, we shall work with only those columns that we find relevant for our model

 We want to use these features to predict the column bad_loan (1 for bad loan, 0 for otherwise)

In [0]:
features = ['grade',                     # grade of the loan (categorical)
            'sub_grade_num',             # sub-grade of the loan as a number from 0 to 1
            'short_emp',                 # one year or less of employment
            'emp_length_num',            # number of years of employment
            'home_ownership',            # home_ownership status: own, mortgage or rent
            'dti',                       # debt to income ratio
            'purpose',                   # the purpose of the loan
            'payment_inc_ratio',         # ratio of the monthly payment to income
            'delinq_2yrs',               # number of delinquincies 
            'delinq_2yrs_zero',          # no delinquincies in last 2 years
            'inq_last_6mths',            # number of creditor inquiries in last 6 months
            'last_delinq_none',          # has borrower had a delinquincy
            'last_major_derog_none',     # has borrower had 90 day or worse rating
            'open_acc',                  # number of open credit accounts
            'pub_rec',                   # number of derogatory public records
            'pub_rec_zero',              # no derogatory public records
            'revol_util',                # percent of available credit being used
           ]
response='bad_loans'

In [0]:
clean_data=loans[features+[response]].dropna()

In [10]:
clean_data.head()

,grade,sub_grade_num,short_emp,emp_length_num,home_ownership,dti,purpose,payment_inc_ratio,delinq_2yrs,delinq_2yrs_zero,inq_last_6mths,last_delinq_none,last_major_derog_none,open_acc,pub_rec,pub_rec_zero,revol_util,bad_loans
0,B,0.4,0,11,RENT,27.65,credit_card,8.14350,0.0,1.0,1.0,1,1,3.0,0.0,1.0,83.7,0
1,C,0.8,1,1,RENT,1.00,car,2.39320,0.0,1.0,5.0,1,1,3.0,0.0,1.0,9.4,1
2,C,1.0,0,11,RENT,8.72,small_business,8.25955,0.0,1.0,2.0,1,1,2.0,0.0,1.0,98.5,0
3,C,0.2,0,11,RENT,20.00,other,8.27585,0.0,1.0,1.0,0,1,10.0,0.0,1.0,21.0,0
4,A,0.8,0,4,RENT,11.20,wedding,5.21533,0.0,1.0,3.0,1,1,9.0,0.0,1.0,28.3,0


**Preprocessing**

One-Hot Encoding for categorical variable.

In [0]:
numerical_cols=['sub_grade_num', 'short_emp', 'emp_length_num','dti', 'payment_inc_ratio', 'delinq_2yrs', \
                'delinq_2yrs_zero', 'inq_last_6mths', 'last_delinq_none', 'last_major_derog_none', 'open_acc',\
                'pub_rec', 'pub_rec_zero','revol_util']

categorical_cols=['grade', 'home_ownership', 'purpose']

In [0]:
mapper = DataFrameMapper([
('grade',sklearn.preprocessing.LabelBinarizer()),
('home_ownership', sklearn.preprocessing.LabelBinarizer()),
('purpose', sklearn.preprocessing.LabelBinarizer()),
])

X1=mapper.fit_transform(clean_data)


X2=np.array(clean_data[numerical_cols])


X = np.hstack((X1,X2)) #Combines X1 and X2 side by side, i.e. stacks them horizontally
y=np.array(clean_data['bad_loans'])

In [0]:
# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=100, stratify=y)

** Logistic Regression model**

In [22]:
log_lm = LogisticRegression()
log_lm.fit(X_train, y_train)
log_lm.score(X_test, y_test)

0.8118464314850066

** Gradient Boosting model**

In [23]:
grd = GradientBoostingClassifier(n_estimators=100)
grd.fit(X_train, y_train)
grd.score(X_test, y_test)

0.8114261699339942

**Decision Tree model**

In [30]:
dtree=DecisionTreeClassifier()
parameters = {'max_depth':[5, 10, 15, 20, 25, 32]}
dtree_gs = GridSearchCV(dtree, parameters, cv=5)
dtree_gs.fit(X_train, y_train)
print(dtree_gs.score(X_test, y_test))

0.8104125979580233


** Random Forest model**

In [24]:
rf=RandomForestClassifier()
parameters = {'max_depth':[5, 15], 'n_estimators':[10,30]}
rf_gs = GridSearchCV(rf, parameters)
rf_gs.fit(X_train, y_train)
print(rf_gs.score(X_test, y_test))

0.8111542359892215


Sample loan application

In [0]:
# Let's try it out on this sample application
a={ 'delinq_2yrs': 0.0,
 'delinq_2yrs_zero': 1.0,
 'dti': 8.7200000000000006,
 'emp_length_num': 0,
 'grade': 'F',
 'home_ownership': 'RENT',
 'inq_last_6mths': 3.0,
 'last_delinq_none': 1,
 'last_major_derog_none': 1,
 'open_acc': 2.0,
 'payment_inc_ratio': 4.5,
 'pub_rec': 0.0,
 'pub_rec_zero': 1.0,
 'purpose': 'vacation',
 'revol_util': 98.5,
 'short_emp': 0,
 'sub_grade_num': 1.0}

In [0]:
def preProcess(a):
    data=list(a.values())
    colz=list(a.keys())
    dfx=pd.DataFrame(data=[data], columns=colz)
    dfx

    XX1=mapper.transform(dfx)
    XX2=dfx[numerical_cols]
    XX = np.hstack((XX1,XX2))
    return XX

In [0]:
XX=preProcess(a)

In [28]:
print(" Accrording to the Logistic Regression model,the probability of loan default is ", log_lm.predict_proba(XX)[:,1][0])

 Accrording to the Logistic Regression model,the probability of loan default is  0.4888497279942286


In [29]:
print(" Accrording to the Grading Boosting model,the probability of loan default is ", grd.predict_proba(XX)[:,1][0])

 Accrording to the Logistic Regression model,the probability of loan default is  0.5343991744106668


In [31]:
print(" Accrording to the Random Forest model,the probability of loan default is ", dtree_gs.predict_proba(XX)[:,1][0])

 Accrording to the Random Forest model,the probability of loan default is  0.3050522648083624
